# Сравнение Transformers и Adapters

## Environment loading

In [ ]:
# !pip install transformers 

In [ ]:
!pip install evaluate datasets wandb adapter-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 88.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 K

In [ ]:
import pandas as pd
import numpy as np
import random

from transformers import pipeline
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

import wandb
wandb.login()
from IPython.display import display, HTML

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
%env WANDB_PROJECT = SentAnalysis_BERT_vs_Adapter

env: WANDB_PROJECT=SentAnalysis_BERT_vs_Adapter


## Loading the dataset

In [ ]:
dataset = load_dataset("ag_news")
dataset["train"][100]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'text': 'Comets, Asteroids and Planets around a Nearby Star (SPACE.com) SPACE.com - A nearby star thought to harbor comets and asteroids now appears to be home to planets, too. The presumed worlds are smaller than Jupiter and could be as tiny as Pluto, new observations suggest.',
 'label': 3}

In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"])

,text,label
0,Inflation rate rises for fourth month in a row The rate of inflation rose in August for the fourth successive month as this year #39;s series of interest rate rises drove up the cost of buying a house.,Business
1,"Microsoft Unveils Windows XP for India (AP) AP - Microsoft Corp. announced Wednesday that it would offer a low-cost, localized version of its Windows XP operating system in India to tap the large market potential in this country of 1 billion people, most of whom do not speak English.",Sci/Tech
2,"Will favorites finally win one? Just why the United States team is considered the favorite in the Ryder Cup is a bit of a mystery, given the simple fact that the European team is 5-3-1 in the last nine matches.",Sports
3,"Job Cuts in Tech Sector Soar, Report Finds NEW YORK (Reuters) - The U.S. technology sector suffered another round of widespread layoffs during the third quarter, with computer firms slashing jobs most aggressively, a report said on Monday.",Sci/Tech
4,Hopes rise for Alitalia rescue Talks between Alitalia and the unions were continuing on Tuesday as hopes rose of staving off bankruptcy at Italy's troubled national airline.,Business
5,"Manufacturing Sector Grows in October NEW YORK - Activity in the nation #39;s manufacturing sector grew in October for the 17th straight month, but at a slower pace than the previous month, the Institute for Supply Management reported Monday.",Business
6,"Bodies Found in Nicaragua May Be Briton, American MANAGUA, Nicaragua (Reuters) - Nicaraguan police found two bodies on Friday on a volcanic island and said they were likely the remains of an American and a Briton who went missing last month.",World
7,"Guantanamo trails set to begin Military court proceedings for four detainees from the US-led war in Afghanistan are due to start on Tuesday at the US naval base in Guantanamo Bay, Cuba.",World
8,"Rough weekend for Big 12 North Going into the season, there were many uncertainties about the quality of the Big 12 North. And questions remain with Missouri, Kansas State and Nebraska suffering upsets this past week.",Sports
9,Sony bags MGM film biz for 4.8B The Japanese electronics giant struck an agreement to buy the fabled movie studio yesterday after rival Time Warner dropped out of the race.,Business


## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers **Tokenizer** which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the **AutoTokenizer.from_pretrained** method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

We pass along **use_fast=True** to the call above to use one of the fast tokenizers (backed by Rust) from the Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in this tutorial if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/120 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the Datasets library to avoid spending time on this step the next time you run your notebook. The Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. Datasets warns you when it uses cached files, you can pass load_from_cache_file=False in the call to map to not use the cached files and force the preprocessing to be applied again.

Note that we passed batched=True to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(3000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

## Transformers model

### Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about sentence classification, we use the AutoModelForSequenceClassification class. Like with the tokenizer, the from_pretrained method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem.

In [ ]:
model_bert = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The warning is telling us we are throwing away some weights (the vocab_transform and vocab_layer_norm layers) and randomly initializing some other (the pre_classifier and classifier layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a Trainer, we will need to define two more things. The most important is the TrainingArguments, which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [ ]:
batch_size = 16

In [ ]:
training_args = TrainingArguments(
    output_dir="test_trainer_transf",
    evaluation_strategy="epoch",
    save_strategy ="epoch",
    optim="adamw_torch",
    # learning_rate=2e-5,
    # weight_decay=0.01,
    # adam_beta1=0.9,
    # adam_beta2=0.999,
    # adam_epsilon=1e-08,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,
    load_best_model_at_end=True,
    report_to="wandb",
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay. Since the best model might not be the one at the end of training, we ask the Trainer to load the best model it saved at the end of training.


The last thing to define for our Trainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [ ]:
accuracy_metric = evaluate.load("accuracy")
recall_metric = evaluate.load('recall')
precision_metric = evaluate.load("precision")
f1_metric = evaluate.load("f1")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)['accuracy']
    recall = recall_metric.compute(predictions=predictions, references=labels, average='micro')['recall']
    precision = precision_metric.compute(predictions=predictions, references=labels, average='micro')['precision']
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='micro')['f1']

    return {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'f1': f1}

You might wonder why we pass along the tokenizer when we already preprocessed our data. This is because we will use it once last time to make all the samples we gather the same length by applying padding, which requires knowing the model's preferences regarding padding (to the left or right? with which token?). The tokenizer has a pad method that will do all of this right for us, and the Trainer will use it. You can customize this part by defining and passing your own data_collator which will receive the samples like the dictionaries seen above and will need to return a dictionary of tensors.

We can now finetune our model by just calling the train method:

In [ ]:
trainer_bert = Trainer(
    model=model_bert,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer_bert.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3000
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1316
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: nedolivko. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,No log,0.313282,0.902000,0.902000,0.902000,0.902000
2,No log,0.325326,0.904000,0.904000,0.904000,0.904000
3,0.314800,0.425356,0.904000,0.904000,0.904000,0.904000
4,0.314800,0.534738,0.892000,0.892000,0.892000,0.892000
5,0.314800,0.543524,0.896000,0.896000,0.896000,0.896000
6,0.052000,0.577541,0.892000,0.892000,0.892000,0.892000
7,0.052000,0.587197,0.893000,0.893000,0.893000,0.893000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to test_trainer_transf/checkpoint-188
Configuration saved in test_trainer_transf/checkpoint-188/config.json
Model weights saved in test_trainer_transf/checkpoint-188/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to test_trainer_transf/checkpoint-376
Configuration saved in test_trainer_transf/checkpoint-376/config.json
Mo

TrainOutput(global_step=1316, training_loss=0.14160134618405512, metrics={'train_runtime': 2159.8053, 'train_samples_per_second': 9.723, 'train_steps_per_second': 0.609, 'total_flos': 5525480991744000.0, 'train_loss': 0.14160134618405512, 'epoch': 7.0})

In [ ]:
trainer_bert.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'eval_loss': 0.31328174471855164,
 'eval_accuracy': 0.902,
 'eval_recall': 0.902,
 'eval_precision': 0.902,
 'eval_f1': 0.902,
 'eval_runtime': 33.7827,
 'eval_samples_per_second': 29.601,
 'eval_steps_per_second': 1.865,
 'epoch': 7.0}

In [ ]:
wandb.finish()

eval/accuracy,▇██▁▃▁▂▇
eval/f1,▇██▁▃▁▂▇
eval/loss,▁▁▄▇▇██▁
eval/precision,▇██▁▃▁▂▇
eval/recall,▇██▁▃▁▂▇
eval/runtime,▄▄▄▁▅▄▃█
eval/samples_per_second,▅▅▅█▄▅▆▁
eval/steps_per_second,▅▅▅█▄▅▆▁
train/epoch,▁▂▃▃▅▆▆▇███
train/global_step,▁▂▃▃▅▆▆▇███
train/learning_rate,█▁


### Adapters model

In [ ]:
from transformers import AutoAdapterModel
from transformers import AdapterTrainer

In [ ]:
%env WANDB_PROJECT = SentAnalysis_BERT_vs_Adapter

env: WANDB_PROJECT=SentAnalysis_BERT_vs_Adapter


In [ ]:
model_adapt = AutoAdapterModel.from_pretrained("bert-base-uncased")

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/re

In [ ]:
adapter_name = model_adapt.load_adapter("AdapterHub/bert-base-uncased-pf-sst2", source="hf")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

Loading module configuration from /root/.cache/huggingface/hub/models--AdapterHub--bert-base-uncased-pf-sst2/snapshots/851ac9789ae6d962cbb47736c786be80cf01059d/adapter_config.json
Adding adapter 'glue_sst2'.
Loading module weights from /root/.cache/huggingface/hub/models--AdapterHub--bert-base-uncased-pf-sst2/snapshots/851ac9789ae6d962cbb47736c786be80cf01059d/pytorch_adapter.bin
Loading module configuration from /root/.cache/huggingface/hub/models--AdapterHub--bert-base-uncased-pf-sst2/snapshots/851ac9789ae6d962cbb47736c786be80cf01059d/head_config.json
Adding head 'glue_sst2' with config {'head_type': 'classification', 'num_labels': 2, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'negative': 0, 'positive': 1}, 'use_pooler': False, 'bias': True}.
Loading module weights from /root/.cache/huggingface/hub/models--AdapterHub--bert-base-uncased-pf-sst2/snapshots/851ac9789ae6d962cbb47736c786be80cf01059d/pytorch_model_head.bin


In [ ]:
model_adapt.set_active_adapters(adapter_name)

In [ ]:
model_adapt.add_classification_head("mrpc", num_labels=5)

Adding head 'mrpc' with config {'head_type': 'classification', 'num_labels': 5, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2, 'LABEL_3': 3, 'LABEL_4': 4}, 'use_pooler': False, 'bias': True}.


In [ ]:
model_adapt.train_adapter(adapter_name)

In [ ]:
training_args =  TrainingArguments(
    output_dir="test_trainer_adapt",
    evaluation_strategy="epoch",
    save_strategy ="epoch",
    optim="adamw_torch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,
    load_best_model_at_end=True,
    report_to="wandb",
)

trainer_adapt = AdapterTrainer(
        model=model_adapt,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
)

PyTorch: setting up devices


In [ ]:
trainer_adapt.train()

The following columns in the training set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: text. If text are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1316
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,No log,0.368143,0.880000,0.880000,0.880000,0.880000
2,No log,0.334343,0.886000,0.886000,0.886000,0.886000
3,0.472200,0.340934,0.895000,0.895000,0.895000,0.895000
4,0.472200,0.336326,0.891000,0.891000,0.891000,0.891000
5,0.472200,0.329582,0.896000,0.896000,0.896000,0.896000
6,0.274700,0.333086,0.895000,0.895000,0.895000,0.895000
7,0.274700,0.328008,0.896000,0.896000,0.896000,0.896000


The following columns in the evaluation set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: text. If text are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
Saving model checkpoint to test_trainer_adapt/checkpoint-188
Configuration saved in test_trainer_adapt/checkpoint-188/glue_sst2/adapter_config.json
Module weights saved in test_trainer_adapt/checkpoint-188/glue_sst2/pytorch_adapter.bin
Configuration saved in test_trainer_adapt/checkpoint-188/glue_sst2/head_config.json
Module weights saved in test_trainer_adapt/checkpoint-188/glue_sst2/pytorch_model_head.bin
Configuration saved in test_trainer_adapt/checkpoint-188/glue_sst2/head_config.json
Module weights saved in test_trainer_adapt/checkpoint-188/glue_sst2/pytorch_model_head.bin
Configuration saved in test_trainer_adapt/checkpoint-188/mrpc/head_config.json
Module weights saved in test_tra

TrainOutput(global_step=1316, training_loss=0.3441855247984541, metrics={'train_runtime': 1647.2538, 'train_samples_per_second': 12.748, 'train_steps_per_second': 0.799, 'total_flos': 5659488543744000.0, 'train_loss': 0.3441855247984541, 'epoch': 7.0})

In [ ]:
trainer_adapt.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertAdapterModel.forward` and have been ignored: text. If text are not expected by `BertAdapterModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


{'eval_loss': 0.3280075788497925,
 'eval_accuracy': 0.896,
 'eval_recall': 0.896,
 'eval_precision': 0.896,
 'eval_f1': 0.8960000000000001,
 'eval_runtime': 34.3794,
 'eval_samples_per_second': 29.087,
 'eval_steps_per_second': 1.832,
 'epoch': 7.0}

In [ ]:
wandb.finish()

eval/accuracy,▁▄█▆████
eval/f1,▁▄█▆████
eval/loss,█▂▃▂▁▂▁▁
eval/precision,▁▄█▆████
eval/recall,▁▄█▆████
eval/runtime,█▅▁▃▅▄█▁
eval/samples_per_second,▁▄█▆▄▅▁█
eval/steps_per_second,▁▄█▆▃▅▂▇
train/epoch,▁▂▃▃▅▆▆▇███
train/global_step,▁▂▃▃▅▆▆▇███
train/learning_rate,█▁
